# Dataset Lending ETL

In [3]:
# Import dependencies
import pandas as pd 
import seaborn as sns 

In [4]:
# import datasets
# accepted applicants data file 
raw_accepted = pd.read_csv('../Archive/accepted_2007_to_2018Q4.csv') 
# rejected applicants data file
raw_rejected = pd.read_csv('../Archive/rejected_2007_to_2018Q4.csv')

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,68407277,NaN,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,68355089,NaN,24700.0,24700.0,24700.0,36 months,11.99,820.28,C,C1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
2,68341763,NaN,20000.0,20000.0,20000.0,60 months,10.78,432.66,B,B4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
3,66310712,NaN,35000.0,35000.0,35000.0,60 months,14.85,829.90,C,C5,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
4,68476807,NaN,10400.0,10400.0,10400.0,60 months,22.45,289.91,F,F1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
#create data copy 
file_rejected = raw_rejected.copy()
file_rejected.head()

,Amount Requested,Application Date,Loan Title,Risk_Score,Debt-To-Income Ratio,Zip Code,State,Employment Length,Policy Code
0,1000.0,2007-05-26,Wedding Covered but No Honeymoon,693.0,10%,481xx,NM,4 years,0.0
1,1000.0,2007-05-26,Consolidating Debt,703.0,10%,010xx,MA,< 1 year,0.0
2,11000.0,2007-05-27,Want to consolidate my debt,715.0,10%,212xx,MD,1 year,0.0
3,6000.0,2007-05-27,waksman,698.0,38.64%,017xx,MA,< 1 year,0.0
4,1500.0,2007-05-27,mdrigo,509.0,9.43%,209xx,MD,< 1 year,0.0


In [7]:
#create data copy 
file_accepted = raw_accepted.copy()
file_accepted.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,68407277,NaN,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,68355089,NaN,24700.0,24700.0,24700.0,36 months,11.99,820.28,C,C1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
2,68341763,NaN,20000.0,20000.0,20000.0,60 months,10.78,432.66,B,B4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
3,66310712,NaN,35000.0,35000.0,35000.0,60 months,14.85,829.90,C,C5,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
4,68476807,NaN,10400.0,10400.0,10400.0,60 months,22.45,289.91,F,F1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
''' 
Only keep those columns that are going to be used for analysis 
removed 
Application Date - date does not play a role in getting rejected or accepted unless you beleive in Numerology :)
Loan Title - For out initial model we have left this out since it did not add a lot of value to the rejected decison. 
Zip Code - We saw people getting accepted and rejected for teh same zipcodes so we think that zipcode does not have a big impact on loan application
Policy Code- This is our target column so we have it but made sure it was all '0'

'''
file_rejected = file_rejected[['Amount Requested', 'Risk_Score',
       'Debt-To-Income Ratio', 'Employment Length']]

In [9]:
file_rejected['Label_target'] = 0
file_rejected.head()

,Amount Requested,Risk_Score,Debt-To-Income Ratio,Employment Length,Label_target
0,1000.0,693.0,10%,4 years,0
1,1000.0,703.0,10%,< 1 year,0
2,11000.0,715.0,10%,1 year,0
3,6000.0,698.0,38.64%,< 1 year,0
4,1500.0,509.0,9.43%,< 1 year,0


In [10]:
file_accepted = file_accepted[['loan_amnt', 'fico_range_low', 'fico_range_high', 'dti', 'emp_length']]
file_accepted.head()

,loan_amnt,fico_range_low,fico_range_high,dti,emp_length
0,3600.0,675.0,679.0,5.91,10+ years
1,24700.0,715.0,719.0,16.06,10+ years
2,20000.0,695.0,699.0,10.78,10+ years
3,35000.0,785.0,789.0,17.06,10+ years
4,10400.0,695.0,699.0,25.37,3 years


In [11]:
file_accepted['average'] = (file_accepted['fico_range_low'] + file_accepted['fico_range_high'])*0.5


In [12]:
file_accepted = file_accepted.drop(columns=['fico_range_low','fico_range_high'], axis=1)

In [13]:
file_accepted['label_target']=1   
file_accepted.head()

,loan_amnt,dti,emp_length,average,label_target
0,3600.0,5.91,10+ years,677.0,1
1,24700.0,16.06,10+ years,717.0,1
2,20000.0,10.78,10+ years,697.0,1
3,35000.0,17.06,10+ years,787.0,1
4,10400.0,25.37,3 years,697.0,1


In [14]:
file_accepted.rename(columns= {'loan_amnt': 'Loan_Amount', 'dti': 'Debt_income_ratio', 'average': 'Credit_Score', 'label_target': 'Loan_Status', 'emp_length':'Emp_Length'}, inplace=True)
file_accepted.head()

,Loan_Amount,Debt_income_ratio,Emp_Length,Credit_Score,Loan_Status
0,3600.0,5.91,10+ years,677.0,1
1,24700.0,16.06,10+ years,717.0,1
2,20000.0,10.78,10+ years,697.0,1
3,35000.0,17.06,10+ years,787.0,1
4,10400.0,25.37,3 years,697.0,1


In [15]:
file_rejected.rename(columns= {'Amount Requested': 'Loan_Amount', 'Debt-To-Income Ratio': 'Debt_income_ratio', 'Risk_Score': 'Credit_Score', 'Label_target': 'Loan_Status', 'Employment Length':'Emp_Length'}, inplace=True)
file_rejected.head()

,Loan_Amount,Credit_Score,Debt_income_ratio,Emp_Length,Loan_Status
0,1000.0,693.0,10%,4 years,0
1,1000.0,703.0,10%,< 1 year,0
2,11000.0,715.0,10%,1 year,0
3,6000.0,698.0,38.64%,< 1 year,0
4,1500.0,509.0,9.43%,< 1 year,0


In [16]:
file_rejected = file_rejected[['Loan_Amount', 'Debt_income_ratio', 'Emp_Length', 'Credit_Score', 'Loan_Status']]
file_rejected.head()

,Loan_Amount,Debt_income_ratio,Emp_Length,Credit_Score,Loan_Status
0,1000.0,10%,4 years,693.0,0
1,1000.0,10%,< 1 year,703.0,0
2,11000.0,10%,1 year,715.0,0
3,6000.0,38.64%,< 1 year,698.0,0
4,1500.0,9.43%,< 1 year,509.0,0


In [17]:
file_rejected['Debt_income_ratio'] = file_rejected['Debt_income_ratio'].str.replace('%','')
file_rejected.head()


,Loan_Amount,Debt_income_ratio,Emp_Length,Credit_Score,Loan_Status
0,1000.0,10,4 years,693.0,0
1,1000.0,10,< 1 year,703.0,0
2,11000.0,10,1 year,715.0,0
3,6000.0,38.64,< 1 year,698.0,0
4,1500.0,9.43,< 1 year,509.0,0


In [18]:
file_rejected['Debt_income_ratio'] = pd.to_numeric(file_rejected['Debt_income_ratio'])
file_rejected.head()

,Loan_Amount,Debt_income_ratio,Emp_Length,Credit_Score,Loan_Status
0,1000.0,10.00,4 years,693.0,0
1,1000.0,10.00,< 1 year,703.0,0
2,11000.0,10.00,1 year,715.0,0
3,6000.0,38.64,< 1 year,698.0,0
4,1500.0,9.43,< 1 year,509.0,0


In [19]:
file_rejected.isnull().sum()

Loan_Amount                 0
Debt_income_ratio           0
Emp_Length             951355
Credit_Score         18497630
Loan_Status                 0
dtype: int64

In [20]:
file_accepted.isnull().sum()

Loan_Amount              33
Debt_income_ratio      1744
Emp_Length           146940
Credit_Score             33
Loan_Status               0
dtype: int64

In [21]:
file_accepted = file_accepted.dropna()
file_accepted.shape

(2113648, 5)

In [22]:
file_rejected = file_rejected.dropna()
file_rejected.shape

(8992595, 5)

In [23]:
#output clean data files for visualization
pd_to_csv('../Archive/accepted_data_clean')
pd_to_csv('../Archive/rejected_data_clean')

NameError: name 'pd_to_csv' is not defined

In [ ]:
consolidated = pd.concat([file_accepted, file_rejected], axis=0).reset_index(drop=True)
consolidated.head()

In [ ]:
consolidated['Emp_Length'].value_counts()

In [ ]:
consolidated['Emp_Length'] = consolidated['Emp_Length'].replace('10+ years','10')
consolidated['Emp_Length'] = consolidated['Emp_Length'].replace('< 1 year','0')
consolidated['Emp_Length'] = consolidated['Emp_Length'].replace('1 year','1')
consolidated['Emp_Length'] = consolidated['Emp_Length'].str.replace(' years','')
consolidated['Emp_Length'] = consolidated['Emp_Length'].replace(' ','')
consolidated['Emp_Length'].value_counts()

In [ ]:
consolidated['Emp_Length'] = pd.to_numeric(consolidated['Emp_Length'])
consolidated.head()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
y = consolidated['Loan_Status']
X = consolidated.drop('Loan_Status',axis=1)

X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=1, stratify=y)

scaler = StandardScaler()

scaler = scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Machine Learning Logistic Regression Modeling

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [ ]:
lr = LogisticRegression(solver='lbfgs',max_iter=100)
lr.fit(X_train_scaled, y_train)

y_pred = lr.predict(X_test_scaled)
print(f'accuracy: {accuracy_score(y_test, y_pred):.3f}')


In [ ]:
confusion_matrix = confusion_matrix(y_test, y_pred)
print(confusion_matrix)

In [ ]:
print(classification_report(y_test, y_pred))